# LLaVA-NEXT Env:
python3.10下pip安装

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
!cd LLaVA
%pip install --upgrade pip  # enable PEP 660 support
%pip install -e .

## Run Code:

In [1]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.6-mistral-7b" # 因内存限制，运行了7b参数模型，如果有条件可尝试："liuhaotian/llava-v1.6-34b" 
model_name = get_model_name_from_path(model_path)

/root/miniconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

class Configs:
    # model_path = "liuhaotian/llava-v1.6-mistral-7b" url
    model_path = "./llava-v1.6-mistral-7b" #本地模型参数
    model_base = None
    model_name = model_name
    image_file = ""
    query = ""
    conv_mode = None
    sep = ","
    temperature = 0.2
    top_p = None
    num_beams = 1
    max_new_tokens = 512
    device = "cuda"

import requests
from PIL import Image
from io import BytesIO
import re

def image_parser(args):
    out = args.image_file.split(args.sep)
    return out

def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

dev = get_device_map()  # 'cpu'
print(dev)
args = Configs()

cuda


In [3]:
test = "./llava-v1.6-mistral-7b/abc.png"

In [4]:
from warnings import filterwarnings
import re
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)


filterwarnings("ignore")
tokenizer, model, image_processor, context_len = load_pretrained_model(
        args.model_path, args.model_base, model_name, load_4bit=True
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.06s/it]


mistral done.


### Multi Page

In [6]:
import os
import json

# imgs_dir
loaded_data = os.listdir("data/docVQA-M")
loaded_data.sort()
loaded_data = loaded_data[:-1]
loaded_data = os.listdir("data/mini_docVQA_multi/images")
loaded_data.sort()

print(loaded_data)
# caculate the number of multipage imgs
# num = 0
# dummy_data = ""
# img_list = []
# for data in loaded_data:
#     if num == 50:
#         break
#     data = data.split('_')[0]
#     if dummy_data != data:
#         dummy_data = data
#         img_list.append(data)
#         num +=1
#print(img_list)

# gt_json = json.load(open("data/docVQA-M/qas/train.json","r+"))['data']
gts = json.load(open('data/mini_docVQA_multi/mini_docVQA_multi.json','r+'))

['ffcn0226_p0.jpg', 'ffcn0226_p1.jpg', 'ffcn0226_p10.jpg', 'ffcn0226_p11.jpg', 'ffcn0226_p12.jpg', 'ffcn0226_p13.jpg', 'ffcn0226_p14.jpg', 'ffcn0226_p15.jpg', 'ffcn0226_p16.jpg', 'ffcn0226_p17.jpg', 'ffcn0226_p18.jpg', 'ffcn0226_p19.jpg', 'ffcn0226_p2.jpg', 'ffcn0226_p20.jpg', 'ffcn0226_p21.jpg', 'ffcn0226_p22.jpg', 'ffcn0226_p23.jpg', 'ffcn0226_p24.jpg', 'ffcn0226_p25.jpg', 'ffcn0226_p26.jpg', 'ffcn0226_p27.jpg', 'ffcn0226_p28.jpg', 'ffcn0226_p29.jpg', 'ffcn0226_p3.jpg', 'ffcn0226_p30.jpg', 'ffcn0226_p31.jpg', 'ffcn0226_p32.jpg', 'ffcn0226_p33.jpg', 'ffcn0226_p34.jpg', 'ffcn0226_p4.jpg', 'ffcn0226_p5.jpg', 'ffcn0226_p6.jpg', 'ffcn0226_p7.jpg', 'ffcn0226_p8.jpg', 'ffcn0226_p9.jpg', 'ffjv0228_p0.jpg', 'ffjv0228_p1.jpg', 'ffjv0228_p10.jpg', 'ffjv0228_p11.jpg', 'ffjv0228_p12.jpg', 'ffjv0228_p2.jpg', 'ffjv0228_p3.jpg', 'ffjv0228_p4.jpg', 'ffjv0228_p5.jpg', 'ffjv0228_p6.jpg', 'ffjv0228_p7.jpg', 'ffjv0228_p8.jpg', 'ffjv0228_p9.jpg', 'fgcn0226_p0.jpg', 'fgcn0226_p1.jpg', 'fgcn0226_p2.jpg', 'f

In [11]:
from llava.conversation import conv_templates, SeparatorStyle
from llava.mm_utils import process_images, tokenizer_image_token
import PIL.Image as Image
import pandas as pd
df = pd.DataFrame(columns=['dataset_name','dataset_split','questionId', 'question','question start','image','true answers','LLAVA answer', 'LLAVA answer match','paddle VQA answer','paddle VQA answer match'])
# loop for muti-pages
for  gt in gts:
    print("-------------------------------------------- Muti page ----------------------------------------------------------------")
    img_file = []
    page_id = gt['answer_page_idx']
    first_word = gt['question'].split(' ')[0].lower()
    for page in gt["page_ids"]:
        img_file.append(os.path.join("data/mini_docVQA_multi/images", page +".jpg"))
    img_file = ','.join(img_file)
    #print(img_file)
    #img_file = os.path.join("data/docVQA-M", question["page_ids"][int(page_id)]+ '.jpg')
    query = "Return the answer only. Do not return any output descriptions or explanations, only the answer: "+ gt["question"]
    answer = gt["answers"]

# model input
    qs = query
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
    if IMAGE_PLACEHOLDER in qs:
        if model.config.mm_use_im_start_end:
            qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
        else:
            qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
    else:
        if model.config.mm_use_im_start_end:
            qs = image_token_se + "\n" + qs
        else:
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print(
            "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
                conv_mode, args.conv_mode, args.conv_mode
            )
        )
    else:
        args.conv_mode = conv_mode

    args.image_file =  img_file
    conv = conv_templates[args.conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    image_files = image_parser(args)
    print(image_files)
    images = load_images(image_files)
    image_sizes = [x.size for x in images]
    print(image_sizes)
    images_tensor = process_images(
        images,
        image_processor,
        model.config
    )
    if not isinstance(images_tensor,torch.Tensor):
        continue
    images_tensor= images_tensor.to(model.device, dtype=torch.float16)
    # print(images_tensor.size())

    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .cuda()
    )

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=images_tensor,
            image_sizes=image_sizes,
            do_sample=True if args.temperature > 0 else False,
            temperature=args.temperature,
            top_p=args.top_p,
            num_beams=args.num_beams,
            max_new_tokens=args.max_new_tokens,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    out_dict = {'dataset_name':'MP-DocVQA','dataset_split':'train', 'question start': first_word, 'questionId':gt['questionId'],'question':qs, 'image': ', '.join(gt['page_ids']),'true answers': ', '.join(gt['answers']), 'LLAVA answer':outputs}
    df.loc[len(df)] = out_dict
    print(out_dict)

-------------------------------------------- Muti page ----------------------------------------------------------------
['data/mini_docVQA_multi/images/xnbl0037_p0.jpg', 'data/mini_docVQA_multi/images/xnbl0037_p1.jpg']
[(1695, 2025), (1695, 2020)]
{'dataset_name': 'MP-DocVQA', 'dataset_split': 'train', 'question start': 'what', 'questionId': 337, 'question': '<image>\nReturn the answer only. Do not return any output descriptions or explanations, only the answer: what is the date mentioned in this letter?', 'image': 'xnbl0037_p0, xnbl0037_p1', 'true answers': '1/8/93', 'LLAVA answer': '1/17/80'}
-------------------------------------------- Muti page ----------------------------------------------------------------
['data/mini_docVQA_multi/images/xnbl0037_p0.jpg', 'data/mini_docVQA_multi/images/xnbl0037_p1.jpg']
[(1695, 2025), (1695, 2020)]
{'dataset_name': 'MP-DocVQA', 'dataset_split': 'train', 'question start': 'what', 'questionId': 338, 'question': '<image>\nReturn the answer only. Do 

In [12]:
# df.to_excel("docVQA-Multi-1-img-output.xlsx",index=False)
df.to_excel("llava_multi_page_VQA_comparasion.xlsx",index=False)